In [111]:
from notion import NotionAPI # type: ignore
import pandas as pd
import numpy_financial as npf
from datetime import date, timedelta
import yfinance as yf
import plotly.express as px
# import pandas_datareader as pdr
import pandas_datareader.data as web

AV_API_KEY = "ILZ89DS0WFHPSG2L"

In [113]:
notion = NotionAPI()
# notion.query_db()
df_tran = notion.get_transactions_df()
df_pos = notion.get_his_positions_df(df_tran)
df_perf = notion.get_performance_df(df_tran, df_pos)
# df_tran
# df_pos

[*********************100%***********************]  4 of 4 completed


,Fecha,Tipo,Producto,ISIN,Bolsa,Centro ejecución,Símbolo,Descripción,Unidades,Valor,Tasa,Ingresado,Retirado,Total,Comprado,Vendido,Dividendos acumulados,Costes acumulados,Efectivo
14,2020-09-28,Ingreso,None,None,None,None,None,Ingreso Sofort/Trustly,NaN,3.000,NaN,3.00,0.0,NaN,0.000,0.0,0.00,0.00,3.000
22,2020-09-29,Ingreso,None,None,None,None,None,Ingreso,NaN,200.000,NaN,203.00,0.0,NaN,0.000,0.0,0.00,0.00,203.000
26,2020-09-30,Compra,iShares Core MSCI World UCITS ETF USD (Acc),IE00B4L5Y983,EAM,XAMS,IWDA,"Compra 1 ISHARES MSCI WOR A@54,59 EUR (IE00B4L...",1.0,54.590,NaN,203.00,0.0,54.590,54.590,0.0,0.00,0.00,148.410
21,2020-09-30,Tasa,None,None,None,None,None,Comisión de conectividad con el mercado 2020 (...,NaN,NaN,-0.51,203.00,0.0,NaN,54.590,0.0,0.00,0.51,147.900
15,2020-09-30,Tasa,None,None,None,None,None,Comisión de compra/venta,NaN,NaN,-0.03,203.00,0.0,NaN,54.590,0.0,0.00,0.54,147.870
25,2020-09-30,Tasa,None,None,None,None,None,Comisión de compra/venta,NaN,NaN,-2.00,203.00,0.0,NaN,54.590,0.0,0.00,2.54,145.870
8,2020-09-30,Compra,iShares Global Clean Energy UCITS ETF USD (Dist),IE00B1XNHC34,XET,XETA,IQQH,Compra 11 iShares Global Clean Energy UCITS ET...,11.0,8.883,NaN,203.00,0.0,97.713,152.303,0.0,0.00,2.54,48.157
10,2020-09-30,Tasa,None,None,None,None,None,Comisión de conectividad con el mercado 2020 (...,NaN,NaN,-0.51,203.00,0.0,NaN,152.303,0.0,0.00,3.05,47.647
30,2020-10-31,Tasa,None,None,None,None,None,Comisión de conectividad con el mercado 2020 (...,NaN,NaN,-0.04,203.00,0.0,NaN,152.303,0.0,0.00,3.09,47.607
32,2020-10-31,Tasa,None,None,None,None,None,Comisión de conectividad con el mercado 2020 (...,NaN,NaN,-0.04,203.00,0.0,NaN,152.303,0.0,0.00,3.13,47.567


In [3]:
# Slicing historic positions dataframe
# df.loc[datetime.today().strftime("%Y-%m-%d")]
row_idx = pd.Timestamp(date.today())
col_idx = pd.IndexSlice[["Cantidad", "Adj Close", "Valor"], ["IWDA.AS", "EMIM.AS"]]
# df.loc[[row_idx], col_idx]
# df.iloc[-31:]["Valor"]
df.columns.get_level_values("Producto").unique()

NameError: name 'df' is not defined

In [ ]:
# df["Valor"].plot.area()
# df["Valor"].columns.name
tickers2names = pd.Series(notion.products["Producto"].values, 
                          index=notion.products["ticker_yfinance"]).to_dict()
df["Valor"].rename(columns=tickers2names)

NameError: name 'df' is not defined

In [ ]:
sel_date = date(2021, 12, 27)
# df_pos.loc["25-12-2021", "Valor"]
a = df_pos.loc[pd.Timestamp(sel_date), "Valor"].sum()
b = df_tran.loc[df_tran["Fecha"] <= pd.Timestamp(sel_date), 
               "Ingresado"].iloc[-1]
a - b

150.4040118865969

In [4]:
# df_tran
# (df_tran.loc[df_tran["Tipo"].isin(["Ingreso", "Retirada"]), ["Fecha","Valor"]]
#         .merge(df_pos["Valor"].sum(axis=1).diff().rename("value_change_daily"),
#                left_on="Fecha", right_index=True, how="right"))
df_mov = df_tran.loc[df_tran["Tipo"].isin(["Ingreso", "Retirada"]), 
                     ["Fecha", "Tipo", "Valor"]]
# df_mov.loc[-1] = [pd.Timestamp(date(2021, 12, 7)), "Retirada", 100.0]
# df_mov
s_mov = df_mov.groupby("Fecha").apply(lambda x: x.loc[x["Tipo"]=="Ingreso", "Valor"].sum() -
                                        x.loc[x["Tipo"]=="Retirada", "Valor"].sum()).rename("movimientos")
s_val_act = df_pos["Valor"].sum(axis=1).rename("valor act.")
s_val_ant = df_pos["Valor"].sum(axis=1).shift().rename("valor ant.")

df_twr = pd.concat([s_val_act, s_val_ant, s_mov], axis=1).fillna(0.0)
df_twr["ROR_daily"] = ((df_twr["valor act."] - (df_twr["valor ant."] + df_twr["movimientos"])) 
                       / (df_twr["valor ant."] + df_twr["movimientos"]) + 1)
df_twr

,valor act.,valor ant.,movimientos,ROR_daily
Fecha,,,,
2020-09-28,3.000000,0.000000,3.0,1.000000
2020-09-29,203.000000,3.000000,200.0,1.000000
2020-09-30,202.674000,203.000000,0.0,0.998394
2020-10-01,204.900000,202.674000,0.0,1.010983
2020-10-02,205.695002,204.900000,0.0,1.003880
...,...,...,...,...
2021-12-25,1621.039012,1621.039012,0.0,1.000000
2021-12-26,1621.039012,1621.039012,0.0,1.000000
2021-12-27,1621.054012,1621.039012,0.0,1.000009


In [97]:
sel_date = date(2021, 12, 27)
# start_date = sel_date - timedelta(days=365*2)
start_date = date(2010, 9, 28)
# date(sel_date.year, 1, 1)

# TWR
(df_twr.loc[start_date:sel_date, "ROR_daily"].cumprod() - 1)[-1] * 100
(df_twr["ROR_daily"].cumprod() - 1) * 100

# # # MWR
# s_mwr = -df_twr.loc[start_date:sel_date, "movimientos"]
# if pd.Timestamp(start_date) < df_twr.index.min():
#     start_date = df_twr.index.min()
# s_mwr[start_date] = - df_twr.loc[pd.Timestamp(start_date), "valor act."]
# s_mwr[sel_date] = df_twr.loc[pd.Timestamp(sel_date), "valor act."] - df_twr.loc[pd.Timestamp(sel_date), "movimientos"]
# # s_mwr
# ((npf.irr(s_mwr) + 1)**365 - 1)*100 # en Excel 19.75%

Fecha
2020-09-28     0.000000
2020-09-29     0.000000
2020-09-30    -0.160591
2020-10-01     0.935961
2020-10-02     1.327587
                ...    
2021-12-25    26.472838
2021-12-26    26.472838
2021-12-27    26.474008
2021-12-28    26.475569
2021-12-29    27.505429
Freq: D, Name: ROR_daily, Length: 458, dtype: float64

In [118]:
# (((df_pos["Adj Close"].pct_change() + 1).cumprod() - 1) * 100)
df_rets = ((df_pos["Adj Close"].pct_change() + 1).join(df_perf["ROR_daily"].rename("Cartera")))

,EMIM.AS,EUNH.DE,IQQH.DE,IWDA.AS,Cartera
Fecha,,,,,
2020-09-28,NaN,NaN,NaN,NaN,0.000000
2020-09-29,-0.657531,0.171443,1.721948,-0.870826,0.000000
2020-09-30,0.966488,-0.085722,5.131171,-0.290275,-0.160591
2020-10-01,1.540838,-0.033543,7.338498,-0.063498,0.935961
2020-10-02,1.441814,0.223622,8.378602,-0.417271,1.327587
...,...,...,...,...,...
2021-12-26,22.855107,-2.124408,24.234371,42.289553,26.472838
2021-12-27,22.855107,-2.113227,24.234371,42.289553,26.474008
2021-12-28,22.855107,-2.098319,24.234371,42.289553,26.475569
